In [1]:
import numpy as np
from datetime import date, timedelta
import pandas as pd
from keras.utils import np_utils
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import LSTM
from keras import callbacks
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.
/Users/prasad/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
df_train = pd.read_csv(
    '/Users/prasad/Desktop/Ecuador/train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    "/Users/prasad/Desktop/Ecuador/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    "/Users/prasad/Desktop/Ecuador/items.csv",
).set_index("item_nbr")


In [4]:
df_train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2016-01-01,25,105574,2.564949,False
1,2016-01-01,25,105575,2.302585,False
2,2016-01-01,25,105857,1.386294,False
3,2016-01-01,25,108634,1.386294,False
4,2016-01-01,25,108701,1.098612,True


In [5]:
df_test.head()

id  onpromotion
store_nbr item_nbr date                              
1         96995    2017-08-16  125497040        False
          99197    2017-08-16  125497041        False
          103501   2017-08-16  125497042        False
          103520   2017-08-16  125497043        False
          103665   2017-08-16  125497044        False

In [6]:
items.head()

,family,class,perishable
item_nbr,,,
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103501,CLEANING,3008,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1


In [7]:
df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train

In [8]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)

In [9]:
promo_2017_train

onpromotion                                              \
date                2017-01-01 2017-01-02 2017-01-03 2017-01-04 2017-01-05   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103520         False      False      False      False      False   
          103665         False      False      False      False      False   
          105574         False      False       True      False      False   
          105575         False      False      False      False      False   
          105577         False      False      False      False      False   
          105693         False      False      False      False      False   
          105737         False      False      False      False      False   
          105857         False      False      False      False      False   
          106716         False      False      False      False      False   
          108079         False      False      False      False      False   
          108634         False      False      False      False      False   
          108696         False      False      False      False      False   
          108698         False      False      False      False      False   
          108701         False      False      False      False      False   
          108786         False      False      False      False      False   
          108797         False      False      False      False      False   
          108831         False      False      False      False      False   
          108862         False      False      False      False      False   
          108952         False      False      False      False      False   
          111223         False      False      False      False      False   
          111397         False      False      False      False      False   
          112830         False      False      False      False      False   
          114778         False      False      False      False      False   
          114790         False      False      False      False      False   
          114799         False      False      False      False      False   
          114800         False      False      False      False      False   
          115267         False      False      False      False      False   
          115611         False      False      False      False      False   
...                        ...        ...        ...        ...        ...   
54        2081056        False      False      False      False      False   
          2081064        False      False      False      False      False   
          2081095        False      False      False      False      False   
          2081142        False      False      False      False      False   
          2081161        False      False      False      False      False   
          2083837        False      False      False      False      False   
          2083859        False      False      False      False      False   
          2084278        False      False      False      False      False   
          2084557        False      False      False      False      False   
          2086882        False      False      False      False      False   
          2087374        False      False      False      False      False   
          2087409        False      False      False      False      False   
          2087543        False      False      False      False      False   
          2087567        False      False      False      False      False   
          2087933        False      False      False      False      False   
          2087978        False      False      False      False      False   
          2088922        False      False      False      False      False   
          2089036        False      False

In [10]:
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)

In [11]:
promo_2017_train.head()

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False        True       False   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False        True       False       False   

date                2017-01-09  2017-01-10     ...      2017-08-06  \
store_nbr item_nbr                             ...                   
1         96995          False       False     ...           False   
          99197          False       False     ...           False   
          103520         False       False     ...           False   
          103665         False       False     ...           False   
          105574         False       False     ...           False   

date                2017-08-07  2017-08-08  2017-08-09  2017-08-10  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-11  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                              
1         96995          False       False       False       False       False  
          99197          False       False       False       False       False  
          103520         False       False       False       False       False  
          103665         False       False       False       False       False  
          105574         False       False       False       False       False  

[5 rows x 227 columns]

In [12]:
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)

In [13]:
promo_2017_test

onpromotion                                              \
date                2017-08-16 2017-08-17 2017-08-18 2017-08-19 2017-08-20   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103501         False      False      False      False      False   
          103520         False      False      False      False      False   
          103665         False      False      False      False      False   
          105574         False      False      False      False      False   
          105575         False      False      False      False      False   
          105576         False      False      False      False      False   
          105577         False      False      False      False      False   
          105693         False      False      False      False      False   
          105737         False      False      False      False      False   
          105857         False      False      False      False      False   
          106716         False      False      False      False      False   
          108079         False      False      False      False      False   
          108634         False      False      False      False      False   
          108696         False      False       True      False      False   
          108698         False      False       True      False      False   
          108701          True      False      False      False      False   
          108786         False      False      False      False      False   
          108797          True       True       True       True       True   
          108831         False      False      False      False      False   
          108833         False      False      False      False      False   
          108862         False      False      False      False      False   
          108952         False      False      False      False      False   
          111223         False      False      False      False      False   
          111397         False      False      False      False      False   
          112830         False      False      False      False      False   
          114778         False      False      False      False      False   
          114790          True       True       True      False       True   
          114799         False      False      False      False      False   
...                        ...        ...        ...        ...        ...   
54        2127921        False      False      False      False      False   
          2127992        False      False      False      False      False   
          2128628        False      False      False      False      False   
          2128799        False      False      False      False      False   
          2129334        False      False      False      False      False   
          2129350        False      False      False      False      False   
          2129387        False      False      False      False      False   
          2129515        False      False      False      False      False   
          2129616        False      False      False      False      False   
          2129678        False      False      False      False      False   
          2129786        False      False      False      False      False   
          2129790        False      False      False      False      False   
          2129892        False      False      False      False      False   
          2129994        False      False      False      False      False   
          2130131        False      False      False      False      False   
          2130219        False      False      False      False      False   
          2130265        False      False      False      False      False   
          2130352        False      False

In [14]:
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)

In [15]:
promo_2017_test.head()

date                2017-08-16  2017-08-17  2017-08-18  2017-08-19  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103501         False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   

date                2017-08-20  2017-08-21  2017-08-22  2017-08-23  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103501         False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   

date                2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103501         False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   

date                2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          103501         False       False       False       False  
          103520         False       False       False       False  
          103665         False       False       False       False

In [16]:
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

In [17]:
promo_2017_test.head()

date                2017-08-16  2017-08-17  2017-08-18  2017-08-19  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-20  2017-08-21  2017-08-22  2017-08-23  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          103520         False       False       False       False  
          103665         False       False       False       False  
          105574         False       False       False       False

In [18]:
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

In [19]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)

In [20]:
df_2017.columns = df_2017.columns.get_level_values(1)

In [21]:
df_2017.head()

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0    0.000000    0.000000    0.000000   
          99197            0.0    0.000000    1.386294    0.693147   
          103520           0.0    0.693147    1.098612    0.000000   
          103665           0.0    0.000000    0.000000    1.386294   
          105574           0.0    0.000000    1.791759    2.564949   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    0.693147    1.098612    0.000000   
          103520      1.098612    1.386294    0.693147    0.000000   
          103665      1.098612    1.098612    0.693147    1.098612   
          105574      2.302585    1.945910    1.609438    1.098612   

date                2017-01-09  2017-01-10     ...      2017-08-06  \
store_nbr item_nbr                             ...                   
1         96995       0.000000    0.000000     ...        1.098612   
          99197       0.000000    0.693147     ...        0.000000   
          103520      0.693147    0.693147     ...        0.000000   
          103665      0.000000    2.079442     ...        0.693147   
          105574      1.386294    2.302585     ...        0.000000   

date                2017-08-07  2017-08-08  2017-08-09  2017-08-10  \
store_nbr item_nbr                                                   
1         96995       1.098612    0.000000    0.000000    0.693147   
          99197       1.098612    0.000000    1.098612    0.000000   
          103520      0.000000    1.386294    0.000000    1.386294   
          103665      1.098612    0.000000    2.079442    2.302585   
          105574      1.791759    2.079442    1.945910    2.397895   

date                2017-08-11  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                              
1         96995       0.000000    0.000000    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    0.000000    0.000000    0.000000  
          103520      0.693147    0.693147    0.693147    0.000000    0.000000  
          103665      1.098612    0.000000    0.000000    0.693147    0.693147  
          105574      1.791759    1.791759    0.000000    1.386294    1.609438  

[5 rows x 227 columns]

In [22]:
items = items.reindex(df_2017.index.get_level_values(1))

In [23]:
items.head()

,family,class,perishable
item_nbr,,,
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1
105574,GROCERY I,1045,0


In [24]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [25]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values
    })
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [26]:
print("Preparing dataset...")
t2017 = date(2017, 5, 31)
X_l, y_l = [], []

Preparing dataset...


In [28]:
X_l

[]

In [29]:
for i in range(6):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)

In [30]:
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

In [32]:
y_train

array([[0.69314718, 0.        , 0.        , ..., 0.        , 0.69314718,
        0.        ],
       [0.69314718, 1.38629436, 1.09861229, ..., 0.69314718, 1.38629436,
        1.09861229],
       [1.38629436, 1.09861229, 1.09861229, ..., 1.09861229, 1.94591015,
        0.69314718],
       ...,
       [0.        , 0.        , 0.        , ..., 0.69314718, 0.        ,
        0.        ],
       [0.69314718, 0.        , 1.09861229, ..., 2.19722458, 1.60943791,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [33]:
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

In [34]:
X_test

,day_1_2017,mean_140_2017,mean_14_2017,mean_30_2017,mean_3_2017,mean_60_2017,mean_7_2017,promo_140_2017,promo_14_2017,promo_60_2017,...,promo_6,promo_7,promo_8,promo_9,promo_10,promo_11,promo_12,promo_13,promo_14,promo_15
0,0.000000,0.153952,0.334438,0.275522,0.000000,0.160866,0.099021,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.000000,0.376532,0.206455,0.331321,0.000000,0.444620,0.156945,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,0.821010,0.573577,0.714515,0.231049,0.756274,0.495105,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.693147,1.040541,1.031388,1.017638,0.462098,1.001216,0.980990,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.609438,1.765433,1.629185,1.714960,0.998577,1.686812,1.560437,24,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2.197225,2.238009,2.382527,2.356156,2.123871,2.303483,2.304131,0,0,0,...,0,0,1,1,1,1,1,1,1,1
6,1.098612,0.602872,0.474556,0.483466,0.597253,0.524279,0.553029,11,4,11,...,0,1,1,1,1,1,1,0,1,1
7,0.693147,0.155165,0.276514,0.258080,0.693147,0.239770,0.553029,7,0,7,...,0,0,0,0,0,0,0,0,1,0
8,0.000000,0.795222,0.681012,0.688510,0.231049,0.634161,0.553029,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2.708050,1.600535,1.619796,1.592271,1.782369,1.648403,1.520775,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
stores_items = pd.DataFrame(index=df_2017.index)
test_ids = df_test[['id']]

In [38]:
items = items.reindex( stores_items.index.get_level_values(1) )

In [39]:
items

,family,class,perishable
item_nbr,,,
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1
105574,GROCERY I,1045,0
105575,GROCERY I,1045,0
105577,GROCERY I,1045,0
105693,GROCERY I,1034,0
105737,GROCERY I,1044,0


In [43]:
X_train = X_train.as_matrix()
X_test = X_test.as_matrix()
X_val = X_val.as_matrix()

In [44]:
X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))[0,0]

array([0.        , 0.07015603, 0.14853154, 0.13862944, 0.23104906,
       0.16369739, 0.29706308, 0.        , 0.        , 0.        ,
       0.        , 0.03465736, 0.1732868 , 0.03465736, 0.1732868 ,
       0.17917595, 0.1732868 , 0.10397208, 0.1732868 , 0.03465736,
       0.1732868 , 0.06931472, 0.1732868 , 0.03465736, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [45]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

In [46]:
model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Dropout(.1))
model.add(Dense(32))
model.add(Dropout(.2))
model.add(Dense(1))
model.compile(loss = 'mse', optimizer='adam', metrics=['mse'])

In [50]:
N_EPOCHS = 5

val_pred = []
test_pred = []
# wtpath = 'weights.hdf5'  # To save best epoch. But need Keras bug to be fixed first.
sample_weights=np.array( pd.concat([items["perishable"]] * 6) * 0.25 + 1 )
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    y = y_train[:, i]
    xv = X_val
    yv = y_val[:, i]
    model.fit(X_train, y, batch_size = 512, nb_epoch = N_EPOCHS, verbose=2,
               sample_weight=sample_weights, validation_data=(xv,yv) ) 
    val_pred.append(model.predict(X_val))
    test_pred.append(model.predict(X_test))

Step 1
Train on 1005090 samples, validate on 167515 samples
Epoch 1/5
31s - loss: 0.4020 - mean_squared_error: 0.3788 - val_loss: 0.3080 - val_mean_squared_error: 0.3080
Epoch 2/5
26s - loss: 0.3539 - mean_squared_error: 0.3349 - val_loss: 0.3030 - val_mean_squared_error: 0.3030
Epoch 3/5
26s - loss: 0.3461 - mean_squared_error: 0.3278 - val_loss: 0.3003 - val_mean_squared_error: 0.3003
Epoch 4/5
26s - loss: 0.3418 - mean_squared_error: 0.3239 - val_loss: 0.2996 - val_mean_squared_error: 0.2996
Epoch 5/5
26s - loss: 0.3391 - mean_squared_error: 0.3214 - val_loss: 0.2967 - val_mean_squared_error: 0.2967
Step 2
Train on 1005090 samples, validate on 167515 samples
Epoch 1/5
26s - loss: 0.3672 - mean_squared_error: 0.3463 - val_loss: 0.3270 - val_mean_squared_error: 0.3270
Epoch 2/5
26s - loss: 0.3628 - mean_squared_error: 0.3423 - val_loss: 0.3306 - val_mean_squared_error: 0.3306
Epoch 3/5
26s - loss: 0.3616 - mean_squared_error: 0.3412 - val_loss: 0.3252 - val_mean_squared_error: 0.3252


29s - loss: 0.4219 - mean_squared_error: 0.3997 - val_loss: 0.3902 - val_mean_squared_error: 0.3902
Epoch 4/5
32s - loss: 0.4210 - mean_squared_error: 0.3989 - val_loss: 0.3888 - val_mean_squared_error: 0.3888
Epoch 5/5
28s - loss: 0.4203 - mean_squared_error: 0.3983 - val_loss: 0.3951 - val_mean_squared_error: 0.3951
Step 13
Train on 1005090 samples, validate on 167515 samples
Epoch 1/5
28s - loss: 0.4153 - mean_squared_error: 0.3920 - val_loss: 0.3818 - val_mean_squared_error: 0.3818
Epoch 2/5
27s - loss: 0.4063 - mean_squared_error: 0.3836 - val_loss: 0.3774 - val_mean_squared_error: 0.3774
Epoch 3/5
28s - loss: 0.4049 - mean_squared_error: 0.3823 - val_loss: 0.3776 - val_mean_squared_error: 0.3776
Epoch 4/5
27s - loss: 0.4044 - mean_squared_error: 0.3818 - val_loss: 0.3764 - val_mean_squared_error: 0.3764
Epoch 5/5
28s - loss: 0.4035 - mean_squared_error: 0.3810 - val_loss: 0.3768 - val_mean_squared_error: 0.3768
Step 14
Train on 1005090 samples, validate on 167515 samples
Epoch 1/

In [51]:
n_public = 5 # Number of days in public test set
weights=pd.concat([items["perishable"]]) * 0.25 + 1
print("Unweighted validation mse: ", mean_squared_error(
    y_val, np.array(val_pred).squeeze(axis=2).transpose()) )
print("Full validation mse:       ", mean_squared_error(
    y_val, np.array(val_pred).squeeze(axis=2).transpose(), sample_weight=weights) )
print("'Public' validation mse:   ", mean_squared_error(
    y_val[:,:n_public], np.array(val_pred).squeeze(axis=2).transpose()[:,:n_public], 
    sample_weight=weights) )
print("'Private' validation mse:  ", mean_squared_error(
    y_val[:,n_public:], np.array(val_pred).squeeze(axis=2).transpose()[:,n_public:], 
    sample_weight=weights) )
    
y_test = np.array(test_pred).squeeze(axis=2).transpose()
df_preds = pd.DataFrame(
    y_test, index=stores_items.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = test_ids.join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lstm.csv', float_format='%.4f', index=None)

('Unweighted validation mse: ', 0.36420061504602463)
('Full validation mse:       ', 0.36364914031005036)
("'Public' validation mse:   ", 0.335986351787782)
("'Private' validation mse:  ", 0.37622313509289956)
